# New York taxis trips

This homework is about New York taxi trips. Here is something from [Todd Schneider](https://toddwschneider.com/posts/analyzing-1-1-billion-nyc-taxi-and-uber-trips-with-a-vengeance/):

> The New York City Taxi & Limousine Commission has released a  detailed historical dataset covering over 1 billion individual taxi trips in the city from January 2009 through December 2019. 
Taken as a whole, the detailed trip-level data is more than just a vast list of taxi pickup and drop off coordinates: it's a story of a City. 
How bad is the rush hour traffic from Midtown to JFK? 
Where does the Bridge and Tunnel crowd hang out on Saturday nights?
What time do investment bankers get to work? How has Uber changed the landscape for taxis?
The dataset addresses all of these questions and many more.

The NY taxi trips dataset has been plowed by series of distinguished data scientists.
The dataset is available from on Amazon S3 (Amazon's cloud storage service).
The link for each file has the following form:

    https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_{year}-{month}.csv

There is one CSV file for each NY taxi service (`yellow`, `green`, `fhv`) and each calendar month (replacing `{year}` and `{month}` by the desired ones).
Each file is moderately large, a few gigabytes. 
The full dataset is relatively large if it has to be handled on a laptop (several hundred gigabytes).

You will focus on the `yellow` taxi service and a pair of months, from year 2015 and from year 2018. 
Between those two years, for hire vehicles services have taken off and carved a huge marketshare.

Whatever the framework you use, `CSV` files prove hard to handle. 
After downloading the appropriate files (this takes time, but this is routine), a first step will consist in converting the csv files into a more Spark friendly format such as `parquet`.

Saving into one of those formats require decisions about bucketing, partitioning and so on. Such decisions influence performance. It is your call.
Many people have been working on this dataset, to cite but a few:


- [1 billion trips with a vengeance](https://toddwschneider.com/posts/analyzing-1-1-billion-nyc-taxi-and-uber-trips-with-a-vengeance/)
- [1 billion trips with R and SQL ](http://freerangestats.info/blog/2019/12/22/nyc-taxis-sql)
- [1 billion trips with redshift](https://tech.marksblogg.com/billion-nyc-taxi-rides-redshift.html)
- [nyc-taxi](https://github.com/fmaletski/nyc-taxi-map)

Depending on your internet connection, **download the files** corresponding to **"yellow" taxis** for the years 2015 and 2018. Download **at least one month** (the same) for 2015 and 2018, if you can download all of them.

**Hint.** The 12 csv for 2015 are about 23GB in total, but the corresponding parquet file, if you can create it for all 12 months, is only about 3GB.

You **might** need the following stuff in order to work with GPS coordinates and to plot things easily.

In [ ]:
!pip install geojson geopandas plotly geopy

In [ ]:
!pip install ipyleaflet

For this homework **we will let you decide on the tools to use** (expected for Spark) and to **find out information all by yourself** (but don't hesitate to ask questions on the `slack` channel).

In [1]:
# import the usual suspects
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import sys
import timeit

%matplotlib inline
import seaborn as sns

sns.set_context("notebook", font_scale=1.2)

# spark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import col
import pyspark.sql.functions as fn
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px



In [2]:
conf = SparkConf()\
    .setAll([ 
        ('spark.app.name', 'Spark NYC taxi trips 2015 2018 study'), 
        ('spark.executor.cores', '16'),
        ('spark.cores.max', '16'), 
        ('spark.driver.memory','4g'),
        ('spark.executor.memory','4g'),
        ('spark.master','local[16]')
])
spark = SparkSession.builder.config(conf=conf).getOrCreate()

spark.sparkContext._conf.getAll()

[('spark.driver.host', '8524e5754722'),
 ('spark.driver.memory', '4g'),
 ('spark.executor.memory', '4g'),
 ('spark.cores.max', '16'),
 ('spark.executor.id', 'driver'),
 ('spark.master', 'local[16]'),
 ('spark.app.name', 'Spark NYC taxi trips 2015 2018 study'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1591330207100'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.executor.cores', '16'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '41697'),
 ('spark.ui.showConsoleProgress', 'true')]

# Loading data as parquet files

We want to organize the data on a per year and per service basis. 
We want to end up with one `parquet` file for each year and each taxi service, since parquet is much better than CSV files.

**Hint.** Depending on your internet connection and your laptop, you can use only the "yellow" service and use one month of 2015 and 2018

CSV files can contain corrupted lines. You may have to work in order to perform ETL (Extract-Transform-Load) in order obtain a properly typed data frame.

You are invited to proceed as follows:

1. Try to read the CSV file without imposing a schema. 
1. Inspect the inferred schema. Do you agree with Spark's typing decision?
1. Eventually correct the schema and read again the data
1. Save the data into parquet files
1. In the rest of your work, **you will only use the parquet files you created**, not the csv files (don't forget to choose a partitioning column and a number of partitions when creating the parquet files).

**Hint.** Don't forget to ask `Spark` to use all the memory and ressources from your computer.

**Hint.** Don't foreget that you should specify a partitioning column and a number of partitions when creating the parquet files.

**Hint.** Note that the schemas of the 2015 and 2018 data are different...

**Hint.** When working on this, ask you and answer to the following questions:

1. What is the `StorageLevel` of the dataframe after reading the csv files?
1. What is the number of partitions of the dataframe? 
1. Is it possible to tune this number at loading time? 
1. Why would we want to modify the number of partitions when creating the parquet files?

In [76]:
input_path = './'
input_file = os.path.join(input_path, 'yellow_tripdata_2015-05.csv')
schema = StructType([
    StructField("VendorID", IntegerType(), True),
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", FloatType(), True),
    StructField("pickup_longitude", DoubleType(), True),
    StructField("pickup_latitude", DoubleType(), True),
    StructField("RateCodeID", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("dropoff_longitude", DoubleType(), True),
    StructField("dropoff_latitude", DoubleType(), True),
    StructField("payment_type", IntegerType(), True),
    StructField("fare_amount", FloatType(), True),
    StructField("extra", FloatType(), True),
    StructField("mta_tax", FloatType(), True),
    StructField("tip_amount", FloatType(), True),
    StructField("tolls_amount", FloatType(), True),
    StructField("improvement_surcharge", FloatType(), True),
    StructField("total_amount", FloatType(), True)
])
df = spark.read\
         .format('csv')\
         .schema(schema)\
         .option("header", "true")\
         .option("mode", "FAILFAST")\
         .option("sep", ",")\
         .load(input_file)

df = df\
    .withColumn("day", fn.dayofweek(col('tpep_pickup_datetime')))\
    .withColumn("month", fn.month(df.tpep_pickup_datetime))\
    .withColumn("year", fn.year(df.tpep_pickup_datetime))

output_path = './'
output_file = os.path.join(output_path, 'yellow_tripdata_2015_05.parquet')
df.write.partitionBy("day")\
        .parquet(output_file)

In [77]:

input_path = './'
input_file = os.path.join(input_path, 'yellow_tripdata_2018-05.csv')
schema = StructType([
    StructField("VendorID", IntegerType(), True),
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", FloatType(), True),
    StructField("RateCodeID", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("PULocationID", IntegerType(), True),
    StructField("DOLocationID", IntegerType(), True),
    StructField("payment_type", IntegerType(), True),
    StructField("fare_amount", FloatType(), True),
    StructField("extra", FloatType(), True),
    StructField("mta_tax", FloatType(), True),
    StructField("tip_amount", FloatType(), True),
    StructField("tolls_amount", FloatType(), True),
    StructField("improvement_surcharge", FloatType(), True),
    StructField("total_amount", FloatType(), True)
])
df = spark.read\
         .format('csv')\
         .schema(schema)\
         .option("header", "true")\
         .option("mode", "FAILFAST")\
         .option("sep", ",")\
         .load(input_file)


df = df\
    .withColumn("day", fn.dayofweek(col('tpep_pickup_datetime')))\
    .withColumn("month", fn.month(df.tpep_pickup_datetime))\
    .withColumn("year", fn.year(df.tpep_pickup_datetime))


output_path = './'
output_file = os.path.join(output_path, 'yellow_tripdata_2018-05.parquet')

df.write.partitionBy("day")\
        .parquet(output_file)

# Investigate (at least) one month of data in 2015

From now on, you will be using **the parquet files you created for 2015**.

We shall visualize several features of taxi traffic during one calendar month
in 2015 and the same calendar month in 2018.

**Hint.** In order to build appealing graphics, you may stick to `matplotlib + seaborn`, you can use also
`plotly`, which is used a lot to build interactive graphics, but you can use whatever you want.

The following longitudes and lattitudes encompass Newark and JFK airports, Northern Manhattan and Verazzano bridge.

In [78]:
long_min = -74.10
long_max = -73.70
lat_min = 40.58
lat_max = 40.90

1. Using these boundaries, **filter the 2015 data** (using pickup and dropoff longitude and latitude) and count the number of trips for each value of `passenger_count` and make a plot of that.

In [79]:
input_path = './'
input_file = os.path.join(input_path, 'yellow_tripdata_2015_05.parquet')

df_15 = spark.read.parquet(input_file)

In [80]:
df_15_trip_count = df_15\
    .filter(df_15.pickup_longitude.between(long_min,long_max))\
    .filter(df_15.pickup_latitude.between(lat_min,lat_max))\
    .groupBy(df_15.passenger_count)\
    .agg(fn.count(col('*')).alias('trip_count'))\
    .orderBy(col('passenger_count').asc())

In [81]:
df_15_trip_count_pd = df_15_trip_count.toPandas()

In [82]:
fig = px.line(df_15_trip_count_pd, x='passenger_count', y='trip_count')
fig.show()

Trips with $0$ or larger than $7$ passengers are pretty rare.
We suspect these to be outliers. 
We need to explore these trips further in order order to understand what might be wrong
with them

1. What's special with trips with zero passengers?
1. What's special with trips with more than $6$ passengers?
1. What is the largest distance travelled during this month? Is it the first taxi on the moon?
1. Plot the distribution of the `trip_distance` (using an histogram for instance) during year 2105. Focus on trips with non-zero trip distance and trip distance less than 30 miles.

In [83]:
# 3
df_15.select(col('trip_distance')).orderBy(col('trip_distance').desc()).limit(1).show()
print('thats for sure the first taxi on the moon')

+-------------+
|trip_distance|
+-------------+
|       1.57E7|
+-------------+

thats for sure the first taxi on the moon


In [84]:
df_15_trip_distance = df_15\
                        .select(col('trip_distance'))\
                        .filter(df_15.trip_distance > 0)\
                        .filter(df_15.trip_distance < 30)\
                        .limit(5000000)

In [85]:
df_15_trip_distance_pd = df_15_trip_distance.toPandas()

In [ ]:
fig = px.histogram(df_15_trip_distance_pd, x="trip_distance",nbins=30)
fig.show()

Let's look at what Spark does for these computations

1. Use the `explain` method or have a look at the [Spark UI](http://localhost:4040/SQL/) to analyze the job. You should be able to assess 
    - Parsed Logical Plan
    - Analyzed Logical Plan
    - Optimized Logical Plan
    - Physical Plan
1. Do the Analyzed Logical Plan and Optimized Logical Plan differ? Spot the differences if any. How would a RDBMS proceed with such a query?
1. How does the physical plan differ from the Optimized Logical Plan? What are the keywords you would not expects in a RDBMS? What is their meaning? 
1. Inspect the stages on [Spark UI](http://localhost:4040/stages/stage). How many *stages* are necessary to complete the Spark job? What are the roles of `HashAggregate` and `Exchange hashpartitioning`?
1. Does the physical plan perform `shuffle` operations? If yes how many?
1. What are tasks with respect to stages (in Spark language)? How many tasks are your stages made of?

In [93]:
# 1

df_15\
    .select(col('trip_distance'))\
    .filter(df_15.trip_distance > 0)\
    .filter(df_15.trip_distance < 30)\
    .limit(5000000).explain("true")  #evaluation of the spark job


== Parsed Logical Plan ==
GlobalLimit 5000000
+- LocalLimit 5000000
   +- Filter (trip_distance#2639 < cast(30 as float))
      +- Filter (trip_distance#2639 > cast(0 as float))
         +- Project [trip_distance#2639]
            +- Relation[VendorID#2635,tpep_pickup_datetime#2636,tpep_dropoff_datetime#2637,passenger_count#2638,trip_distance#2639,pickup_longitude#2640,pickup_latitude#2641,RateCodeID#2642,store_and_fwd_flag#2643,dropoff_longitude#2644,dropoff_latitude#2645,payment_type#2646,fare_amount#2647,extra#2648,mta_tax#2649,tip_amount#2650,tolls_amount#2651,improvement_surcharge#2652,total_amount#2653,month#2654,year#2655,day#2656] parquet

== Analyzed Logical Plan ==
trip_distance: float
GlobalLimit 5000000
+- LocalLimit 5000000
   +- Filter (trip_distance#2639 < cast(30 as float))
      +- Filter (trip_distance#2639 > cast(0 as float))
         +- Project [trip_distance#2639]
            +- Relation[VendorID#2635,tpep_pickup_datetime#2636,tpep_dropoff_datetime#2637,passenger_c

In [94]:
# 2

# there are a diffrence bitween the analyzed logical plan, and the optimized logical plan
# in the exemple below, in the optimized logical plan, spark sees that there 
# is no need for two filters, it do the job with only one filter using & operator

df_15.filter(df_15.trip_distance > 0)\
    .filter(df_15.trip_distance < 30).explain("true")

== Parsed Logical Plan ==
'Filter (trip_distance#2639 < 30)
+- Filter (trip_distance#2639 > cast(0 as float))
   +- Relation[VendorID#2635,tpep_pickup_datetime#2636,tpep_dropoff_datetime#2637,passenger_count#2638,trip_distance#2639,pickup_longitude#2640,pickup_latitude#2641,RateCodeID#2642,store_and_fwd_flag#2643,dropoff_longitude#2644,dropoff_latitude#2645,payment_type#2646,fare_amount#2647,extra#2648,mta_tax#2649,tip_amount#2650,tolls_amount#2651,improvement_surcharge#2652,total_amount#2653,month#2654,year#2655,day#2656] parquet

== Analyzed Logical Plan ==
VendorID: int, tpep_pickup_datetime: timestamp, tpep_dropoff_datetime: timestamp, passenger_count: int, trip_distance: float, pickup_longitude: double, pickup_latitude: double, RateCodeID: int, store_and_fwd_flag: string, dropoff_longitude: double, dropoff_latitude: double, payment_type: int, fare_amount: float, extra: float, mta_tax: float, tip_amount: float, tolls_amount: float, improvement_surcharge: float, total_amount: float,

In [ ]:
# 3

# Physical Plan is generated once Optimized Logical Plan is created
# It generates different kinds of execution strategies

In [ ]:
# 4

# spark need 16 stages to complete the job

# HashAggregate is essential to build a 
# TungstenAggregationIterator to process 
# the aggregation for a given partition

# Exchange hashpartitioning


Now, compute the following and produce relevant plots:

1. Break down the trip distance distribution for each day of week
1. Count the number of distinct pickup location
1. Compute and display tips and profits as a function of the pickup location

In [ ]:
#1
df_15_trip_distance_per_day = df_15\
                                .select(df_15.day,df_15.trip_distance)\
                                .filter(df_15.trip_distance > 0)\
                                .filter(df_15.trip_distance < 30)\
                                .limit(5000000)

In [ ]:
df_15_trip_distance_per_day_pd = df_15_trip_distance_per_day.toPandas() 

In [ ]:
fig = make_subplots(rows=3, cols=3)
day=1
for i in range(1,4):
    for j in range(1,4):
        fig.add_trace(
            go.Histogram(
                x=df_15_trip_distance_per_day_pd[(df_15_trip_distance_per_day_pd['day'] == day)].trip_distance, 
                nbinsx=30,
                name=str(day)
            ),
            row=1, col=1
        )
        day=day+1

fig.show()

In [ ]:
# 2
pickup_dist_nbr = df_15.select(df_15.pickup_longitude,df_15.pickup_latitude).distinct().count()

print('the number of distinct pickup location is '+str(pickup_dist_nbr))

In [ ]:
# 3
df_15\
    .groupBy(df_15.pickup_longitude,df_15.pickup_latitude)\
    .agg(
        fn.round(fn.sum(col('tip_amount')),2).alias('total_tips_per_pickup_location'),
        fn.round(fn.sum(col('fare_amount')+col('extra')+col('mta_tax')+col('improvement_surcharge')),2).alias('profits')
    )\
    .show()

# Investigate one month of trips data in 2015 and 2018

 Consider one month of trips data from `yellow` taxis for each year

1. Filter and cache/persist the result

## Assessing seasonalities and looking at time series

Compute and plot the following time series indexed by day of the week and hour of day:

1. The number of pickups
1. The average fare
1. The average trip duration
1. Plot the average number of ongoing trips

## Rides to the airports

In order to find the longitude and lattitude of JFK and Newark airport as well as the longitude and magnitudes 
of Manhattan, you can use a service like [geojson.io](http://geojson.io/).
Plot the following time series, indexed the day of the week and hour of the day

1. Median duration of taxi trip leaving Midtown (Southern Manhattan) headed for JFK Airport
1. Median taxi duration of trip leaving from JFK Airport to Midtown (Southern Manhattan)

## Geographic information

For this, you will need to find tools to display maps and to build choropeth maps.
We let you look and find relevant tools to do this.

1. Build a heatmap where color is a function of
    1. number of `pickups`
    2. number of `dropoffs`
    3. number of `pickups` with dropoff at some airport (JFK, LaGuardia, Newark)
2. Build a choropeth map where color is a function of
    1. number of pickups in the area
    1. ratio of number of payments by card/number of cash payments for pickups in the area
    2. ratio of total fare/trip duration for dropoff in the area
3. Build an interactive chorophet with a slider allowing the user to select an `hour of day` and where the color is a function of
    1. average number of dropoffs in the area during that hour the day
    2. average ratio of tip over total fare amount for pickups in the area at given hour of the day

In [4]:
# 4 Filter and cache/persist the result
input_path = './'
input_file = os.path.join(input_path, 'yellow_tripdata_2015_05.parquet')

df_15 = spark.read.parquet(input_file)

input_path = './'
input_file = os.path.join(input_path, 'yellow_tripdata_2018-05.parquet')

df_18 = spark.read.parquet(input_file)

In [5]:
df_15 = df_15.withColumn("hour",fn.hour(col('tpep_pickup_datetime')))
df_18 = df_18.filter(df_18.year == 2018).withColumn("hour",fn.hour(col('tpep_pickup_datetime')))

In [6]:
df_15.persist(storageLevel = StorageLevel.MEMORY_ONLY_SER)
df_18.persist(storageLevel = StorageLevel.MEMORY_ONLY_SER)

DataFrame[VendorID: int, tpep_pickup_datetime: timestamp, tpep_dropoff_datetime: timestamp, passenger_count: int, trip_distance: float, RateCodeID: int, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: int, fare_amount: float, extra: float, mta_tax: float, tip_amount: float, tolls_amount: float, improvement_surcharge: float, total_amount: float, month: int, year: int, day: int, hour: int]

In [26]:
# 4.1  Assessing seasonalities and looking at time series

def plot_4_1_charts(df,dayOrhour,yAxisname):
    color=['indianred','lightsalmon']
    year=[2015,2018]
    df_pd = df.toPandas()
    fig = go.Figure()
    for i in range(2):
        fig.add_trace(go.Bar(
            x=df_pd[(df_pd['year']==year[i])][dayOrhour],
            y=df_pd[(df_pd['year']==year[i])][yAxisname],
            name=year[i],
            marker_color=color[i]
        )) 
    fig.update_layout(barmode='group',title="The "+yAxisname+" indexed by "+dayOrhour)
    fig.show()

In [19]:
# 1 The number of pickups
df_15_day_pickup_nbr = df_15.groupby(df_15.year,df_15.day).agg(fn.count(col('*')).alias('pickup_nbr'))
df_18_day_pickup_nbr = df_18.groupby(df_18.year,df_18.day).agg(fn.count(col('*')).alias('pickup_nbr'))

df_day_pickup_nbr = df_15_day_pickup_nbr.union(df_18_day_pickup_nbr)

plot_4_1_charts(df_day_pickup_nbr,'day','pickup_nbr')

In [20]:
df_15_hour_pickup_nbr = df_15.groupby(col('year'),col('hour')).agg(fn.count(col('*')).alias('pickup_nbr'))

df_18_hour_pickup_nbr = df_18.groupby(col('year'),col('hour')).agg(fn.count(col('*')).alias('pickup_nbr'))

df_hour_pickup_nbr = df_15_hour_pickup_nbr.union(df_18_hour_pickup_nbr)

plot_4_1_charts(df_hour_pickup_nbr,'hour','pickup_nbr')

In [21]:
# 4.1 - 2 The average fare

# per day
df_15_day_avg_fare = df_15.groupby(col('year'),col('day')).agg(fn.round(fn.avg(col('fare_amount')),2).alias('avg_fare'))

df_18_day_avg_fare = df_18.groupby(col('year'),col('day')).agg(fn.round(fn.avg(col('fare_amount')),2).alias('avg_fare'))

df_day_avg_fare = df_15_day_avg_fare.union(df_18_day_avg_fare)

plot_4_1_charts(df_day_avg_fare,'day','avg_fare')

In [22]:
# per hour
df_15_hour_avg_fare = df_15.groupby(col('year'),col('hour')).agg(fn.round(fn.avg(col('fare_amount')),2).alias('avg_fare'))

df_18_hour_avg_fare = df_18.groupby(col('year'),col('hour')).agg(fn.round(fn.avg(col('fare_amount')),2).alias('avg_fare'))

df_hour_avg_fare = df_15_hour_avg_fare.union(df_18_hour_avg_fare)

plot_4_1_charts(df_hour_avg_fare,'hour','avg_fare')

In [23]:
# 4.1 - 3 the average trip duration

df_15_day_avg_trip_duration = df_15\
                                .withColumn('trip_duration',col('tpep_dropoff_datetime').cast('long')-col('tpep_pickup_datetime').cast('long'))\
                                .groupby(col('year'),col('day'))\
                                .agg(fn.round(fn.avg(col('trip_duration')),2).alias('avg_trip_duration'))

df_18_day_avg_trip_duration = df_18\
                                .withColumn('trip_duration',col('tpep_dropoff_datetime').cast('long')-col('tpep_pickup_datetime').cast('long'))\
                                .groupby(col('year'),col('day'))\
                                .agg(fn.round(fn.avg(col('trip_duration')),2).alias('avg_trip_duration'))
    
df_day_avg_trip_duration = df_15_day_avg_trip_duration.union(df_18_day_avg_trip_duration)

plot_4_1_charts(df_day_avg_trip_duration,'day','avg_trip_duration')

In [31]:
# per hour
df_15_hour_avg_trip_duration = df_15\
                                .withColumn('trip_duration',col('tpep_dropoff_datetime').cast('long')-col('tpep_pickup_datetime').cast('long'))\
                                .groupby(col('year'),col('hour'))\
                                .agg(fn.round(fn.avg(col('trip_duration')),2).alias('avg_trip_duration'))

df_18_hour_avg_trip_duration = df_18\
                                .withColumn('trip_duration',col('tpep_dropoff_datetime').cast('long')-col('tpep_pickup_datetime').cast('long'))\
                                .filter(col('trip_duration') > 0)\
                                .groupby(col('year'),col('hour'))\
                                .agg(fn.round(fn.avg(col('trip_duration')),2).alias('avg_trip_duration'))

df_hour_avg_trip_duration = df_15_hour_avg_trip_duration.union(df_18_hour_avg_trip_duration)

plot_4_1_charts(df_hour_avg_trip_duration,'hour','avg_trip_duration')

In [57]:
# 4.2  Rides to the airports
# 1 midtown to jfk 
magic_percentile = fn.expr('percentile_approx(trip_duration, 0.5)')

df_15_day_median_mid_jfk = df_15\
                    .filter(col('dropoff_longitude').between(-73.79199028015137,-73.77735614776611))\
                    .filter(col('dropoff_latitude').between(40.64092123848905,40.64912697157757))\
                    .filter(col('pickup_longitude').between(-74.01060104370116,-73.95927429199219))\
                    .filter(col('pickup_latitude').between(40.736201792985945,40.77235187838098))\
                    .withColumn('trip_duration',col('tpep_dropoff_datetime').cast('long')-col('tpep_pickup_datetime').cast('long'))\
                    .groupby(col('year'),col('day'))\
                    .agg(magic_percentile.alias('median'))
df_18_day_median_mid_jfk = df_18\
                    .filter(col('DOLocationID') == 132 )\
                    .filter(col('PULocationID').isin([246,50,68,48,90,186,230,163,161,164,234,162,170,107,229,233,137,224]))\
                    .withColumn('trip_duration',col('tpep_dropoff_datetime').cast('long')-col('tpep_pickup_datetime').cast('long'))\
                    .groupby(col('year'),col('day'))\
                    .agg(magic_percentile.alias('median'))

df_day_median_mid_jfk = df_15_day_median_mid_jfk.union(df_18_day_median_mid_jfk)
plot_4_1_charts(df_day_median_mid_jfk,'day','median')

In [56]:
# per hour
magic_percentile = fn.expr('percentile_approx(trip_duration, 0.5)')

df_15_hour_median_mid_jfk = df_15\
                    .filter(col('dropoff_longitude').between(-73.79199028015137,-73.77735614776611))\
                    .filter(col('dropoff_latitude').between(40.64092123848905,40.64912697157757))\
                    .filter(col('pickup_longitude').between(-74.01060104370116,-73.95927429199219))\
                    .filter(col('pickup_latitude').between(40.736201792985945,40.77235187838098))\
                    .withColumn('trip_duration',col('tpep_dropoff_datetime').cast('long')-col('tpep_pickup_datetime').cast('long'))\
                    .groupby(col('year'),col('hour'))\
                    .agg(magic_percentile.alias('median'))
df_18_hour_median_mid_jfk = df_18\
                    .filter(col('DOLocationID') == 132 )\
                    .filter(col('PULocationID').isin([246,50,68,48,90,186,230,163,161,164,234,162,170,107,229,233,137,224]))\
                    .withColumn('trip_duration',col('tpep_dropoff_datetime').cast('long')-col('tpep_pickup_datetime').cast('long'))\
                    .groupby(col('year'),col('hour'))\
                    .agg(magic_percentile.alias('median'))

df_hour_median_mid_jfk = df_15_hour_median_mid_jfk.union(df_18_hour_median_mid_jfk)
plot_4_1_charts(df_hour_median_mid_jfk,'hour','median')

In [60]:
# 1 jfk to midtown  
magic_percentile = fn.expr('percentile_approx(trip_duration, 0.5)')

df_15_day_median_jfk_mid = df_15\
                    .filter(col('dropoff_longitude').between(-74.01060104370116,-73.95927429199219))\
                    .filter(col('dropoff_latitude').between(40.736201792985945,40.77235187838098))\
                    .filter(col('pickup_longitude').between(-73.79199028015137,-73.77735614776611))\
                    .filter(col('pickup_latitude').between(40.64092123848905,40.64912697157757))\
                    .withColumn('trip_duration',col('tpep_dropoff_datetime').cast('long')-col('tpep_pickup_datetime').cast('long'))\
                    .groupby(col('year'),col('day'))\
                    .agg(magic_percentile.alias('median'))
df_18_day_median_jfk_mid = df_18\
                    .filter(col('PULocationID') == 132)\
                    .filter(col('DOLocationID').isin([246,50,68,48,90,186,230,163,161,164,234,162,170,107,229,233,137,224] ))\
                    .withColumn('trip_duration',col('tpep_dropoff_datetime').cast('long')-col('tpep_pickup_datetime').cast('long'))\
                    .groupby(col('year'),col('day'))\
                    .agg(magic_percentile.alias('median'))

df_day_median_jfk_mid = df_15_day_median_jfk_mid.union(df_18_day_median_jfk_mid)
plot_4_1_charts(df_day_median_jfk_mid,'day','median')

In [61]:
# per hour
magic_percentile = fn.expr('percentile_approx(trip_duration, 0.5)')

df_15_hour_median_jfk_mid = df_15\
                    .filter(col('pickup_longitude').between(-73.79199028015137,-73.77735614776611))\
                    .filter(col('pickup_latitude').between(40.64092123848905,40.64912697157757))\
                    .filter(col('dropoff_longitude').between(-74.01060104370116,-73.95927429199219))\
                    .filter(col('dropoff_latitude').between(40.736201792985945,40.77235187838098))\
                    .withColumn('trip_duration',col('tpep_dropoff_datetime').cast('long')-col('tpep_pickup_datetime').cast('long'))\
                    .groupby(col('year'),col('hour'))\
                    .agg(magic_percentile.alias('median'))
df_18_hour_median_jfk_mid = df_18\
                    .filter(col('PULocationID') == 132 )\
                    .filter(col('DOLocationID').isin([246,50,68,48,90,186,230,163,161,164,234,162,170,107,229,233,137,224]))\
                    .withColumn('trip_duration',col('tpep_dropoff_datetime').cast('long')-col('tpep_pickup_datetime').cast('long'))\
                    .groupby(col('year'),col('hour'))\
                    .agg(magic_percentile.alias('median'))

df_hour_median_jfk_mid = df_15_hour_median_jfk_mid.union(df_18_hour_median_jfk_mid)
plot_4_1_charts(df_hour_median_jfk_mid,'hour','median')

In [62]:
df_15.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'pickup_longitude',
 'pickup_latitude',
 'RateCodeID',
 'store_and_fwd_flag',
 'dropoff_longitude',
 'dropoff_latitude',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'month',
 'year',
 'day',
 'hour']